In [3]:
import xgboost
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.metrics import mean_squared_error
import numpy as np
from scipy.stats import spearmanr
from sklearn.linear_model import LinearRegression

In [43]:
x_train = pd.read_csv('./global_data/x_train.csv')
y_train = pd.read_csv('./global_data/y_train.csv')
x_test = pd.read_csv('./global_data/x_test.csv')

In [44]:
x_train_fr = x_train[x_train['COUNTRY'] == 'FR'].drop('COUNTRY', axis=1)
y_train_fr = y_train.merge(x_train_fr)['TARGET']

In [47]:
x_train_fr = x_train_fr.fillna(0)

In [48]:
def getLr(x, y): 
    xTr, xTe, yTr, yTe = train_test_split(x, y, test_size=0.2, random_state=42) 

    lr = LinearRegression()
    lr.fit(xTr, yTr)

    print('Spearman correlation for train set {:.1f}%'.format(100 * spearmanr(lr.predict(xTe), yTe).correlation))

In [49]:
getLr(x_train_fr, y_train_fr)

Spearman correlation for train set 1.1%
